In [4]:
#name - Rahul Bansal
# BCA
# 5th semester 
# Rollnumber - 210934106340
#QR Code Scanner minor project

In [5]:
# Import necessary libraries
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import webbrowser

In [6]:
# Set to store opened URLs
opened_urls = set()

def open_url_in_browser(url):
    """
    Function to open a URL in the default web browser.

    Parameters:
    - url (str): The URL to be opened in the browser.
    """
    # Check if the URL has not been opened before
    if url not in opened_urls:
        webbrowser.open(url)
        opened_urls.add(url)

def is_point_inside_polygon(point, polygon):
    """
    Function to check if a point is inside a polygon.

    Parameters:
    - point (tuple): The point coordinates (x, y).
    - polygon (list): List of polygon vertices.

    Returns:
    - bool: True if the point is inside the polygon, False otherwise.
    """
    polygon = np.array(polygon, dtype=int)
    result = cv2.pointPolygonTest(polygon, point, False)
    return result == 1

def scan_qr_code(image, mouse_position):
    """
    Function to scan QR codes in an image and display QR code data as a tooltip.

    Parameters:
    - image (numpy.ndarray): The input image to scan for QR codes.
    - mouse_position (tuple): The current mouse position (x, y).

    Returns:
    - List[Dict[str, Any]]: A list of dictionaries containing information about each QR code found.
    Each dictionary has keys 'data' (the QR code content) and 'rect' (the bounding box coordinates).
    """
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use the decode function from pyzbar to find QR codes in the image
    qr_codes = decode(gray)

    # List to store information about each QR code found
    qr_code_info = []

    # Loop through the detected QR codes
    for code in qr_codes:
        # Extract QR code data and bounding box coordinates
        qr_data = code.data.decode('utf-8')
        qr_rect = code.polygon if len(code.polygon) > 0 else code.rect

        # Append information to the list
        qr_code_info.append({
            'data': qr_data,
            'rect': qr_rect
        })

        # Draw a rectangle or polyline around the QR code on the original image
        if len(qr_rect) == 2:  # If it's a rectangle
            cv2.rectangle(image, qr_rect[0], qr_rect[1], (0, 255, 0), 2)
        elif len(qr_rect) == 4:  # If it's a polygon
            cv2.polylines(image, [np.array(qr_rect, dtype=int)], isClosed=True, color=(0, 255, 0), thickness=2)

        # Check if the mouse is hovering over the QR code
        if is_point_inside_polygon(mouse_position, qr_rect):
            # Display QR code data as a tooltip
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, qr_data, (mouse_position[0] + 10, mouse_position[1] - 10),
                        font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

            # Check if the QR code contains a URL
            if qr_data.startswith('http://') or qr_data.startswith('https://'):
                cv2.putText(image, "Press 'O' to open in browser", (mouse_position[0] + 10, mouse_position[1] + 20),
                            font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    return qr_code_info

def main():
    # Open the default camera (camera index 0)
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        # Read a frame from the camera
        ret, frame = cap.read()

        if not ret:
            print("Failed to capture frame")
            break

        # Get the mouse position
        mouse_position = (cv2.getTrackbarPos('X', 'QR Code Scanner'), cv2.getTrackbarPos('Y', 'QR Code Scanner'))

        # Call the function to scan QR codes in the frame
        qr_code_info = scan_qr_code(frame, mouse_position)

        # Display the frame with QR codes highlighted
        cv2.imshow('QR Code Scanner', frame)

        # Check for the 'ESC' key to exit the loop
        key = cv2.waitKey(1)
        if key == 27:  # ASCII code for 'ESC'
            break
        elif key == ord('o') or key == ord('O'):
            # Open the URL in the browser for the hovered QR code
            for qr_info in qr_code_info:
                if qr_info['data'].startswith('http://') or qr_info['data'].startswith('https://'):
                    open_url_in_browser(qr_info['data'])

    # Release the camera and close the OpenCV window
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Create a window for the QR Code Scanner
    cv2.namedWindow('QR Code Scanner')
    cv2.createTrackbar('X', 'QR Code Scanner', 0, 1920, lambda x: x)
    cv2.createTrackbar('Y', 'QR Code Scanner', 0, 1080, lambda x: x)
    main()